<a href="https://colab.research.google.com/github/johansbustamante-gif/Proyecto-Inteligencia-Artificial/blob/main/02%20-%20preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==========================================
# 🧹 Limpieza y procesado sencillo en Colab
# (con detección segura de formatos de fecha)
# ==========================================
import pandas as pd
import numpy as np
import warnings
from scipy import stats
from google.colab import drive

warnings.filterwarnings("ignore", category=FutureWarning)

# Montar Google Drive
print("🔗 Montando Google Drive...")
drive.mount('/content/drive')
print("✅ Drive montado.\n")

# Rutas (ajusta si hace falta)
input_path = '/content/drive/MyDrive/DataProyectoIA/train.csv'
output_path = '/content/drive/MyDrive/DataProyectoIA/train_limpio.csv'

# Función auxiliar: intentar varios formatos comunes sin warnings
def try_parse_date_series(s: pd.Series, min_parsed_frac=0.5):
    """
    Intenta parsear la Series `s` usando formatos comunes; devuelve parsed Series
    si alguno parsea >= min_parsed_frac, sino devuelve None.
    """
    if s.dropna().empty:
        return None

    formats = [
        "%Y-%m-%d", "%d/%m/%Y", "%m/%d/%Y", "%Y/%m/%d",
        "%d-%m-%Y", "%Y.%m.%d", "%d %b %Y", "%d %B %Y",
        "%Y%m%d", "%d%m%Y"
    ]
    sample = s.dropna().astype(str)
    # pruebo formatos explícitos (evita warnings de infer)
    for fmt in formats:
        parsed = pd.to_datetime(sample, format=fmt, errors='coerce')
        frac = parsed.notna().mean()
        if frac >= min_parsed_frac:
            # reconstruyo con mismos índices que s (incluye NaN)
            full_parsed = pd.to_datetime(s.astype(str), format=fmt, errors='coerce')
            return full_parsed
    # si ningún formato fijo funcionó, no forzamos parseo (evitamos dateutil/warnings)
    return None

# Función principal de limpieza
def simple_clean_process_safe_dates(input_path,
                                    output_path=None,
                                    drop_col_threshold=0.9,
                                    row_na_thresh=0.5,
                                    outlier_method='iqr',
                                    iqr_multiplier=1.5,
                                    fill_categorical_with_mode=True,
                                    fill_unknown_label="__DESCONOCIDO__",
                                    date_min_parsed_frac=0.5,
                                    verbose=True):
    if verbose: print(f"📂 Leyendo archivo: {input_path}")
    df = pd.read_csv(input_path)
    if verbose: print(f"   → Shape inicial: {df.shape}")

    # 1) Eliminar columnas con muchos NaN
    col_nan_ratio = df.isna().mean()
    cols_to_drop = col_nan_ratio[col_nan_ratio > drop_col_threshold].index.tolist()
    if cols_to_drop:
        if verbose: print(f"🧹 Eliminando {len(cols_to_drop)} columnas con >{drop_col_threshold*100:.0f}% NaN")
        df = df.drop(columns=cols_to_drop)
    else:
        if verbose: print("✅ No se eliminaron columnas por NaN excesivo.")

    # 2) Duplicados
    before = len(df)
    df = df.drop_duplicates()
    if verbose: print(f"🧩 Duplicados eliminados: {before - len(df)}")

    # 3) Eliminar filas con muchos NaN
    row_nan_ratio = df.isna().mean(axis=1)
    rows_to_drop = row_nan_ratio[row_nan_ratio > row_na_thresh].index
    if len(rows_to_drop) > 0:
        if verbose: print(f"🚮 Eliminando {len(rows_to_drop)} filas con >{row_na_thresh*100:.0f}% NaN")
        df = df.drop(index=rows_to_drop)
    else:
        if verbose: print("✅ No se eliminaron filas por NaN excesivo.")

    # 4) Intentar parsear fechas de manera segura (sin warnings)
    obj_cols = df.select_dtypes(include=['object']).columns.tolist()
    for c in obj_cols[:]:
        try:
            parsed = try_parse_date_series(df[c], min_parsed_frac=date_min_parsed_frac)
            if parsed is not None:
                if verbose: print(f"📅 Columna '{c}' convertida a datetime (>= {date_min_parsed_frac*100:.0f}% parseada).")
                df[c] = parsed
        except Exception:
            # si algo falla, simplemente no convertimos esa columna
            pass

    # 5) Separar tipos y mostrar
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = df.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
    if verbose: print(f"🔢 Numéricas: {len(num_cols)} | Categóricas: {len(cat_cols)}")

    # 6) Imputación
    for c in num_cols:
        miss = df[c].isna().sum()
        if miss:
            med = df[c].median()
            df[c] = df[c].fillna(med)
            if verbose: print(f"  • Num '{c}': {miss} NaN → mediana={med}")

    for c in cat_cols:
        miss = df[c].isna().sum()
        if miss:
            if fill_categorical_with_mode:
                try:
                    mode = df[c].mode(dropna=True).iloc[0]
                except Exception:
                    mode = fill_unknown_label
            else:
                mode = fill_unknown_label
            df[c] = df[c].fillna(mode)
            if verbose: print(f"  • Cat '{c}': {miss} NaN → '{mode}'")

    # 7) Outliers por IQR (elimina filas con outlier en cualquier num_col)
    if outlier_method == 'iqr' and num_cols:
        if verbose: print("⚖️ Tratando outliers por IQR (eliminando filas con outlier en cualquier columna numérica).")
        to_drop = set()
        for c in num_cols:
            q1, q3 = df[c].quantile([0.25, 0.75])
            iqr = q3 - q1
            low, high = q1 - iqr_multiplier * iqr, q3 + iqr_multiplier * iqr
            mask = (df[c] < low) | (df[c] > high)
            if mask.any():
                if verbose: print(f"  • {c}: {mask.sum()} outliers fuera de [{low:.3f}, {high:.3f}]")
                to_drop.update(df[mask].index.tolist())
        if to_drop:
            df = df.drop(index=list(to_drop))
            if verbose: print(f"🗑️ Filas eliminadas por outliers: {len(to_drop)}")
        else:
            if verbose: print("✅ No se detectaron outliers por IQR.")
    else:
        if verbose: print("⚠️ No se aplica tratamiento de outliers o no hay columnas numéricas.")

    df = df.reset_index(drop=True)
    if verbose:
        print(f"✅ Shape final: {df.shape}")
        print(f"💾 Guardando resultado en: {output_path}")

    if output_path:
        df.to_csv(output_path, index=False)

    print("🎯 Limpieza finalizada.")
    return df

# Ejecutar limpieza
df_clean = simple_clean_process_safe_dates(
    input_path=input_path,
    output_path=output_path,
    drop_col_threshold=0.9,
    row_na_thresh=0.5,
    outlier_method='iqr',
    iqr_multiplier=1.5,
    fill_categorical_with_mode=True,
    date_min_parsed_frac=0.5,
    verbose=True
)










🔗 Montando Google Drive...
Mounted at /content/drive
✅ Drive montado.

📂 Leyendo archivo: /content/drive/MyDrive/DataProyectoIA/train.csv
   → Shape inicial: (692500, 21)
✅ No se eliminaron columnas por NaN excesivo.
🧩 Duplicados eliminados: 0
🚮 Eliminando 1639 filas con >50% NaN
🔢 Numéricas: 6 | Categóricas: 15
  • Cat 'E_VALORMATRICULAUNIVERSIDAD': 4648 NaN → 'Entre 1 millón y menos de 2.5 millones'
  • Cat 'E_HORASSEMANATRABAJA': 29218 NaN → 'Más de 30 horas'
  • Cat 'F_ESTRATOVIVIENDA': 30498 NaN → 'Estrato 2'
  • Cat 'F_TIENEINTERNET': 24990 NaN → 'Si'
  • Cat 'F_EDUCACIONPADRE': 21539 NaN → 'Secundaria (Bachillerato) completa'
  • Cat 'F_TIENELAVADORA': 38134 NaN → 'Si'
  • Cat 'F_TIENEAUTOMOVIL': 41984 NaN → 'No'
  • Cat 'E_PAGOMATRICULAPROPIO': 4859 NaN → 'No'
  • Cat 'F_TIENECOMPUTADOR': 36464 NaN → 'Si'
  • Cat 'F_TIENEINTERNET.1': 24990 NaN → 'Si'
  • Cat 'F_EDUCACIONMADRE': 22025 NaN → 'Secundaria (Bachillerato) completa'
⚖️ Tratando outliers por IQR (eliminando filas con o